In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('training_data.csv')
NUM_STOCKS = len(df)
ORIGINAL_NUM_DAYS = len(df.iloc[0]) - 1
batch = 50
# last_day = ORIGINAL_NUM_DAYS  - (ORIGINAL_NUM_DAYS %  batch)

df.pop('ticker')
df = df.T
df

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
from sklearn.preprocessing import MinMaxScaler

main = []
for i in range(84):
  main.append(df.filter([i]).values)

main

In [ ]:
X,y = [],[]
for stock in main:
  for i in range(batch,ORIGINAL_NUM_DAYS):
    X.append(stock[i-batch:i,0])
    y.append(stock[i,0])
X = np.array(X)
X=X.reshape((X.shape[0],X.shape[1],1))

y = np.array(y)

X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

X_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM,Dropout,Dense,Input

model = Sequential([Input((batch,1)),
                    LSTM(50),
                    Dense(32, activation = 'relu'),
                    Dense(32, activation = 'relu'),
                    Dense(1)])

model.compile(loss='mse', optimizer='adam', metrics=['mean_absolute_error'])

model.fit(X_train,y_train, validation_data=(X_valid,y_valid), epochs=5)




In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_test, predictions, squared=False)
rms

In [ ]:
xdf = pd.DataFrame()
xdf['actual'] = y_test[234:632]
xdf['prediction'] = predictions[234:632]

plt.figure(figsize = (16,8))
plt.plot(xdf[['actual','prediction']])
plt.legend(['actual', 'pred'])

plt.show()


In [ ]:
xdf

In [ ]:
model.save('model', save_format="h5")